In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import *

## Simple Custom Dense Layer

In [2]:
class SimpleDense(layers.Layer):
    def __init__(self, units=32):
        super(SimpleDense, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name='kernel', initial_value=w_init(shape=(input_shape[-1], self.units),
                             dtype='float32'),trainable=True)
        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name='bias', initial_value=b_init(shape=self.units, dtype='float32'), trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

### Basic Demo

In [3]:
inputs = tf.constant([[5]], dtype='float32')
inputs

Metal device set to: Apple M1


2022-01-31 08:11:53.143617: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-31 08:11:53.143850: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[5.]], dtype=float32)>

In [4]:
custom_dense = SimpleDense(units=1)

In [5]:
custom_dense(inputs)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.2344516]], dtype=float32)>

In [6]:
custom_dense.variables

[<tf.Variable 'simple_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.04689032]], dtype=float32)>,
 <tf.Variable 'simple_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

## Load and Preprocess Data

In [7]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

In [8]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

## Training and Evaluation with Simple Dense

In [9]:
model = Sequential([layers.Input(shape=(28, 28)),
                    layers.Flatten(),
                    SimpleDense(128),
                    layers.Lambda(lambda x : tf.maximum(x, 0.0)),
                    layers.Dropout(0.1),
                    SimpleDense(64),
                    layers.Lambda(lambda x : tf.maximum(x, 0.0)),
                    layers.Dropout(0.1),
                    layers.Dense(10, activation='softmax')])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 simple_dense_1 (SimpleDense  (None, 128)              100480    
 )                                                               
                                                                 
 lambda (Lambda)             (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 simple_dense_2 (SimpleDense  (None, 64)               8256      
 )                                                               
                                                                 
 lambda_1 (Lambda)           (None, 64)                0

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='acc')
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5


2022-01-31 08:11:53.671677: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-31 08:11:53.802650: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1872/1875 [============================>.] - ETA: 0s - loss: 0.2862 - acc: 0.9133

2022-01-31 08:12:02.733783: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 10s 5ms/step - loss: 0.2861 - acc: 0.9133 - val_loss: 0.1365 - val_acc: 0.9563
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1275 - acc: 0.9616 - val_loss: 0.1020 - val_acc: 0.9674
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0932 - acc: 0.9709 - val_loss: 0.0772 - val_acc: 0.9748
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0758 - acc: 0.9755 - val_loss: 0.0805 - val_acc: 0.9752
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0662 - acc: 0.9789 - val_loss: 0.0712 - val_acc: 0.9791


## Custom Dense Layer with Activation

In [14]:
class CustomDense(layers.Layer):
    def __init__(self, units=32, activation=None):
        super(CustomDense, self).__init__()
        self.units = units
        self.activation = activations.get(activation)
        
    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name='kernel', initial_value=w_init(shape=(input_shape[-1], self.units),
                             dtype='float32'),trainable=True)
        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name='bias', initial_value=b_init(shape=self.units, dtype='float32'), trainable=True)
    
    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.w) + self.b)

## Training and Evaluation with Activated Custom Dense

In [15]:
model = Sequential([layers.Input(shape=(28, 28)),
                    layers.Flatten(),
                    CustomDense(128, activation='relu'),
                    layers.Dropout(0.1),
                    CustomDense(64, activation='relu'),
                    layers.Dropout(0.1),
                    layers.Dense(10, activation='softmax')])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 custom_dense_1 (CustomDense  (None, 128)              100480    
 )                                                               
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 custom_dense_2 (CustomDense  (None, 64)               8256      
 )                                                               
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 10)               

In [17]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='acc')
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
  21/1875 [..............................] - ETA: 9s - loss: 2.0454 - acc: 0.3765 

2022-01-31 08:13:38.448105: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1867/1875 [============================>.] - ETA: 0s - loss: 0.2846 - acc: 0.9147

2022-01-31 08:13:47.442005: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 10s 5ms/step - loss: 0.2843 - acc: 0.9149 - val_loss: 0.1239 - val_acc: 0.9616
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1273 - acc: 0.9616 - val_loss: 0.0883 - val_acc: 0.9719
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0938 - acc: 0.9713 - val_loss: 0.0795 - val_acc: 0.9749
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0773 - acc: 0.9759 - val_loss: 0.0708 - val_acc: 0.9792
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0654 - acc: 0.9796 - val_loss: 0.0715 - val_acc: 0.9783
